In [1]:
%env HSA_OVERRIDE_GFX_VERSION=10.3.0
%env ROCM_PATH=/opt/rocm


env: HSA_OVERRIDE_GFX_VERSION=10.3.0
env: ROCM_PATH=/opt/rocm


In [2]:
from textwrap import wrap
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#TODO document jupyter
import pickle
import configparser
import sqlite3 as sl
import pandas as pd
import numpy as np

configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)
datasetPathDatabase =  configParser.get('COMMON', 'datasetPathDatabase') + '/dataset.db'
image_size =  int(configParser.get('COMMON', 'resizeImageTo'))
audio_length_used =  configParser.get('train_imagen', 'audio_length_used') 
model_filename =  configParser.get('train_imagen', 'model_filename') + '_'  + audio_length_used +  's.pt'
sub_epochs=  int(configParser.get('train_imagen', 'sub_epochs') )
batch_size=  int(configParser.get('train_imagen', 'batch_size') )
sample_every=  int(configParser.get('train_imagen', 'sample_every') ) - 1
save_model_every=  int(configParser.get('train_imagen', 'save_model_every') ) - 1
epochs=  int(configParser.get('train_imagen', 'epochs') )

continue_from_epoch = int(configParser.get('train_imagen', 'continue_from_epoch'))
continue_from_offset = int(configParser.get('train_imagen', 'continue_from_offset'))
continue_from_epoch_and_offset_flag = int(configParser.get('train_imagen', 'continue_from_epoch_and_offset_flag'))
db_chunk = int(configParser.get('train_imagen', 'db_chunk'))

con = sl.connect(datasetPathDatabase)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None



In [3]:
def speaker_emb_preprocess(speaker_emb2):

    speaker_emb2 = pickle.loads(speaker_emb2)
    speaker_emb2 = speaker_emb2.squeeze()
    speaker_emb2 = np.pad(speaker_emb2, (288), 'constant', constant_values=(0))
    speaker_emb2 = np.tile(speaker_emb2, (1, 1))
    speaker_emb2 = speaker_emb2 / 200.0
    speaker_emb2 = np.array(speaker_emb2).tolist()
    return speaker_emb2


def audio_emb_preprocess2(speaker_emb2):
    speaker_emb2 = pickle.loads(speaker_emb2)
    a = speaker_emb2 #np.zeros(shape=(24, 512))
    b = np.zeros(shape=(a.shape[0], 768-a.shape[1]))
    c = np.zeros(shape=(250-a.shape[0], 768))
    arr = np.concatenate((a, b), axis=1)
    arr = np.concatenate((arr, c), axis=0)
    arr = arr / 10.0
    speaker_emb2 = np.array(arr).tolist()
    return speaker_emb2

from PIL import Image
def getImage(face_path):
    im = Image.open(face_path)
    im.load() # required for png.split()

    im2 = Image.new("RGB", im.size, (255, 255, 255))
    im2.paste(im, mask=im.split()[3]) # 3 is the alpha channel
    im3 = np.array(im2)
    im4 = np.rollaxis(im3,2)
    return im4

In [4]:
import random
def process_gender(gender):
    if(random.random() < 0.2):
        return np.zeros(768)
    elif(gender == 'man'):
        return np.ones(768)
    elif(gender == "woman"):
        return np.ones(768) * -1
    else:
        return np.zeros(768)
    
# TODO
def process_ethnicity(eth):
    x = np.zeros(768)
    if(random.random() < 0.2):
        x = x
    elif(eth == "indian"):
        x[0] = 1
    elif(eth == "asian"):
        x[16]=1
    elif(eth == "latino hispanic"):
        x[2]=1
    elif(eth == "black"):
        x[3]=1
    elif(eth == "middle eastern"):
        x[4]=1
    elif(eth == "white"):
        x[5]=1 
    else:
        x = x
    return x     

def process_language(lan):
    x = np.zeros(768)
    if(random.random() < 0.2):
        x = x
    elif(lan == "Arabic"):
        x[0] = 1
    elif(lan == "Portuguese"):
        x[16]=1
    elif(lan == "Romansh_Sursilvan"):
        x[2]=1
    elif(lan == "Japanese"):
        x[3]=1
    elif(lan == "Ukranian"):
        x[4]=1
    elif(lan == "German"):
        x[5]=1   
    elif(lan == "Chinese_China"):
        x[6]=1   
    elif(lan == "Welsh"):
        x[7]=1  
    elif(lan == "English"):
        x[8]=1
    elif(lan == "Kabyle"):
        x[9]=1 
    elif(lan == "Kyrgyz"):
        x[10]=1
    elif(lan == "Georgian"):
        x[11]=1
    elif(lan == "Persian"):
        x[12]=1 
    elif(lan == "French"):
        x[13]=1
    elif(lan == "Interlingua"):
        x[14]=1
    elif(lan == "Swedish"):
        x[15]=1
    elif(lan == "Spanish"):
        x[16]=1 
    elif(lan == "Dhivehi"):
        x[17]=1
    elif(lan == "Kinyarwanda"):
        x[18]=1 
    elif(lan == "Tatar"):
        x[19]=1
    elif(lan == "Hakha_Chin"):
        x[20]=1 
    elif(lan == "Tamil"):
        x[21]=1 
    elif(lan == "Greek"):
        x[22]=1
    elif(lan == "Latvian"):
        x[23]=1 
    elif(lan == "Russian"):
        x[24]=1
    elif(lan == "Breton"):
        x[25]=1
    elif(lan == "Catalan"):
        x[26]=1    
    elif(lan == "Maltese"):
        x[27]=1 
    elif(lan == "Slovenian"):
        x[28]=1    
    elif(lan == "Indonesian"):
        x[29]=1    
    elif(lan == "Dutch"):
        x[30]=1
    elif(lan == "Chinese_Taiwan"):
        x[31]=1 
    elif(lan == "Sakha"):
        x[32]=1 
    elif(lan == "Polish"):
        x[33]=1 
    elif(lan == "Czech"):
        x[34]=1 
    elif(lan == "Romanian"):
        x[35]=1 
    elif(lan == "Mangolian"):
        x[36]=1 
    elif(lan == "Italian"):
        x[37]=1 
    elif(lan == "Chinese_Hongkong"):
        x[38]=1 
    elif(lan == "Estonian"):
        x[39]=1 
    elif(lan == "Basque"):
        x[40]=1 
    elif(lan == "Esperanto"):
        x[41]=1 
    elif(lan == "Frisian"):
        x[42]=1 
    elif(lan == "Turkish"):
        x[43]=1 
    elif(lan == "Chuvash"):
        x[44]=1 
    else:
        x = x
    return x       

In [5]:
#TODO add age
def process_input(ethnicity,gender,language,speaker_emb,audio_emb):
    speaker_emb = np.asarray(speaker_emb, dtype=np.float32)
    speaker_emb = speaker_emb.squeeze()
    audio_emb = np.asarray(audio_emb, dtype=np.float32)
    audio_emb = audio_emb.squeeze()
    #print(speaker_emb.shape)
    h = np.vstack((ethnicity, gender))
    h = np.vstack((h, language))
    h = np.vstack((h, speaker_emb))
    h = np.vstack((h, audio_emb))

    j = np.zeros(768)
    j = np.tile(j,(256-h.shape[0],1))
    h = np.vstack((h, j))
    #print(h.shape)
    return h.tolist()

from PIL import Image
def process_image_path(path):
    
    image = Image.open(path).convert('RGB')
    
    pix = np.array(image,np.float32)
    pix = np.moveaxis(pix, -1, 0)
    
    pix = pix / 255
    return pix.tolist()

In [ ]:
import multiprocessing
import train_imagen_batch



range_of_epochs = range(1,epochs + 1)

if(continue_from_epoch_and_offset_flag != 0):
    
    range_of_epochs = range(continue_from_epoch, epochs + 1)

offset_zero_flag = 0

for epoch in range_of_epochs:

    print("Training in epoch " + str(epoch))

    offset = 0

    if(continue_from_epoch_and_offset_flag != 0 and offset_zero_flag == 0):
        offset = continue_from_offset
        offset_zero_flag = 1

    while(True):

        print("Training in offset " + str(offset))

        data = con.execute("SELECT V.ID, V.VIDEO_PATH, V.AGE, " + 
                            "V.ETHNICITY CAPTION_E, " +
                            "lower(V.GENDER) CAPTION_G, " +
                            "A.SPEAKER_EMB, "+ "A.WAV_TO_VEC, " +
                            "A.LANG CAPTION_L, "+
                            "F.FACE_PATH "+
                            "FROM VIDEO V "+
                            "INNER JOIN AUDIO A ON V.ID = A.VIDEO_ID INNER JOIN FACE F ON V.ID = F.VIDEO_ID WHERE AUDIO_PRE = 2 AND FACES_PRE = 1 AND AUDIO_LENGTH = " + audio_length_used + ' ' +
                            "LIMIT "+ str(db_chunk) +" OFFSET " + str(offset))
        dataGotten = data.fetchall()

        if(len(dataGotten) == 0):
            break


        df = pd.DataFrame(dataGotten,columns = ['ID','VIDEO_PATH','AGE','caption_e','caption_g','SPEAKER_EMB','AUDIO_EMB','caption_l','image_path'])

        df3 = df[["image_path","caption_e","caption_g","caption_l"]]

        data_frame = df3
        data_frame['SPEAKER_EMB'] = df['SPEAKER_EMB']
        data_frame['AUDIO_EMB'] = df['AUDIO_EMB']


        for index, row in data_frame.iterrows():
            x = speaker_emb_preprocess(data_frame.loc[index,"SPEAKER_EMB"])
            x = [x]
            data_frame.loc[index,"SPEAKER_EMB"] = x

        #CHECK
        for index, row in data_frame.iterrows():
            x = audio_emb_preprocess2(data_frame.loc[index,"AUDIO_EMB"])
            x = [x]
            #AADFS = AADFS
            data_frame.loc[index,"AUDIO_EMB"] = x

        for index, row in data_frame.iterrows():
            x = process_gender(data_frame.loc[index,"caption_g"])
            x = [x]
            #AADFS = AADFS
            data_frame.loc[index,"caption_g"] = x

        for index, row in data_frame.iterrows():
            x = process_language(data_frame.loc[index,"caption_l"])
            x = [x]
            #AADFS = AADFS
            data_frame.loc[index,"caption_l"] = x
            
        for index, row in data_frame.iterrows():
            x = process_ethnicity(data_frame.loc[index,"caption_e"])
            x = [x]
            #AADFS = AADFS
            data_frame.loc[index,"caption_e"] = x

        data_frame['INPUT'] = data_frame['SPEAKER_EMB']

        for index, row in data_frame.iterrows():
            x = process_input(data_frame.loc[index,"caption_e"],data_frame.loc[index,"caption_g"],
                            data_frame.loc[index,"caption_l"],data_frame.loc[index,"SPEAKER_EMB"],
                            data_frame.loc[index,"AUDIO_EMB"])
            x = [x]
            #AADFS = AADFS
            data_frame.loc[index,"INPUT"] = x

        data_frame = data_frame.drop(['caption_e', 'caption_g','caption_l','SPEAKER_EMB','AUDIO_EMB'], axis=1)

        for index, row in data_frame.iterrows():
            x = process_image_path(data_frame.loc[index,"image_path"])
            x = [x]
            #AADFS = AADFS
            data_frame.loc[index,"image_path"] = x

        input = data_frame['INPUT'].to_numpy()
        input = np.array([np.array(xi) for xi in input])




        output = data_frame['image_path'].to_numpy()
        output = np.array([np.array(xi) for xi in output])
        output.squeeze().shape




        proc = multiprocessing.Process(target=train_imagen_batch.train_batch_unet1, args=(input,output,model_filename,sub_epochs,batch_size,sample_every,save_model_every,image_size,))
        proc.start()
        proc.join()

        proc = multiprocessing.Process(target=train_imagen_batch.train_batch_unet2, args=(input,output,model_filename,sub_epochs,batch_size,sample_every,save_model_every,image_size,))
        proc.start()
        proc.join()
        offset = offset + db_chunk

/home/gamal/anaconda3/envs/ds2f_m_i/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Training in epoch 1
Training in offset 0
Training Unet No. 1


2023-05-29 10:02:44.412954: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-29 10:02:44.654442: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


The base dimension of your u-net should ideally be no smaller than 128, as recommended by a professional DDPM trainer https://nonint.com/2022/05/04/friends-dont-let-friends-train-small-diffusion-models/
training with dataset of 975 samples and validating with randomly splitted 25 samples
valid loss: 0.3891371190547943
unet 2 has not been trained
when sampling, you can pass stop_at_unet_number to stop early in the cascade, so it does not try to generate with untrained unets


sampling loop time step: 100%|██████████| 1000/1000 [00:48<00:00, 20.83it/s]
0it [00:48, ?it/s]


...

sampling loop time step: 100%|██████████| 1000/1000 [00:39<00:00, 25.11it/s]
0it [00:39, ?it/s]


valid loss: 0.0029951126780360937
valid loss: 0.0044032796286046505
valid loss: 0.0035991535987704992
valid loss: 0.0031399105209857225
valid loss: 0.002632175339385867
valid loss: 0.003924978896975517
valid loss: 0.006749325897544622
valid loss: 0.0029421267099678516
valid loss: 0.003143544076010585
valid loss: 0.0027985458727926016
valid loss: 0.004610040690749884
valid loss: 0.0047305431216955185
valid loss: 0.00443777022883296
valid loss: 0.0024474661331623793
valid loss: 0.0030717928893864155
valid loss: 0.003143091918900609
valid loss: 0.0039881812408566475
valid loss: 0.0019507108954712749
valid loss: 0.00314798834733665
valid loss: 0.006973410490900278


0it [00:00, ?it/s]

KeyboardInterrupt: 

sampling loop time step:  35%|███▌      | 351/1000 [00:14<00:26, 24.40it/s]

In [ ]:
con.close()